In [126]:
from nupack import *

In [127]:
model = Model(material='dna', celsius=37, sodium=0.5, magnesium=0.1)

In [128]:
model

In [129]:
# Define sequence domains
dx = Domain('N16', name='dx')
dx1m = Domain('N16', name='dx1m')
dx2m = Domain('N16', name='dx2m')
dy1 = Domain('N16', name='dy1')
dy2 = Domain('N16', name='dy2')
dw = Domain('N16', name='dw')

# Define strands containing these domains
sx = TargetStrand([dx], name='sx')
sx1m = TargetStrand([dx1m], name='sx1m')
sx2m = TargetStrand([dx2m], name='sx2m')
sy1 = TargetStrand([dy1], name='sy1')
sy2 = TargetStrand([dy2], name='sy2')
sw = TargetStrand([dw], name='sw')
base1 = TargetStrand([~dy1, ~dx],  name='base1')
base2 = TargetStrand([~dy2, ~dx1m], name='base2')
base3 = TargetStrand([~dw, ~dx2m], name='base3')


C1 = TargetComplex([base1, sy1], 'D16(U16+)', name='C1')
C2 = TargetComplex([base2, sy2], 'D16(U16+)', name='C2')
C3 = TargetComplex([base3, sw], 'D16(U16+)', name='C3')
cx = TargetComplex([sx], 'U16', name='cx')
cx1m = TargetComplex([sx1m], 'U16', name='cx1m')
cx2m = TargetComplex([sx2m], 'U16', name='cx2m')
cy1 = TargetComplex([sy1], 'U16', name='cy1')
cy2 = TargetComplex([sy2], 'U16', name='cy2')
cw = TargetComplex([sw], 'U16', name='cw')

# Hard constraints
d_8_3 = Diversity(word=8, types=3)
d_10_4 = Diversity(word=10, types=3)
p_4 = Pattern(['A4', 'G4', 'C4', 'T4'])

ssm4 = SSM(word=6, weight=0.5)
diff1 = EnergyMatch([dx, dx1m, dx2m])
diff2 = EnergyMatch([dy1, dy2])

t1 = TargetTube(on_targets={cx: 1e-7, cx1m: 1e-7, cx2m: 1e-7, cy1: 1e-7, cy2: 1e-7, cw: 1e-7, C1:1e-7, C2: 1e-7, C3: 1e-7}, name='t1',
               off_targets=SetSpec(max_size=3, exclude=[[base1,sx,sy1], [base2,sx1m,sy2], [base3,sx2m,sw], [base1,sx], [base2,sx1m], [base3,sx2m]]))
t2 = TargetTube(on_targets={cx: 1e-7, cx1m: 1e-7, cx2m: 1e-7, cy1: 1e-7, cy2: 1e-7, cw: 1e-7}, name='t2',
                off_targets=SetSpec(max_size=2))

# Set a stop condition of 1% and a seed for random number generation to get a reproducible result for this demo
my_options = DesignOptions(f_stop=0.01, seed=93)

# Define and run the test tube design job
my_design = tube_design(tubes=[t1], model=model, options=my_options, hard_constraints=[d_8_3, d_10_4, p_4], soft_constraints=[diff1, diff2, ssm4])
my_results = my_design.run(trials=3)[0]

# Display the design results
my_results

Domain,Sequence
dw,GGAAATTAGAGCTGGC
dw*,GCCAGCTCTAATTTCC
dx,GCGGCAGATAAATAGG
dx*,CCTATTTATCTGCCGC
dx1m,AGGCCCGAGAATTAAA
dx1m*,TTTAATTCTCGGGCCT
dx2m,GTAGGCTTGGTGATTG
dx2m*,CAATCACCAAGCCTAC
dy1,GTTACATCTCCATCCG
dy1*,CGGATGGAGATGTAAC


In [130]:
strands = list(my_results.to_analysis.strands.values())
keys = list(my_results.to_analysis.strands.keys())

In [131]:
data = {}
for strand in strands:
    data[strand] = 1e-7

an1 = Tube(strands=data, complexes=SetSpec(max_size=2), name='an1')

In [132]:
tube_results = tube_analysis(tubes=[an1], model=model)

In [133]:
tube_results

Complex,Pfunc,ΔG (kcal/mol)
(base1),7.7025e+1,-2.677
(base2),1.9347e+2,-3.245
(base3),8.1757e+0,-1.295
(sw),1.1743e+0,-0.099
(sx),1.4094e+0,-0.212
(sx1m),1.4638e+0,-0.235
(sx2m),1.6458e+0,-0.307
(sy1),1.4451e+0,-0.227
(sy2),1.2643e+0,-0.145
(base1+base1),1.2897e+8,-11.510


In [134]:
domains = my_results.to_analysis.domains
for k,v in domains.items():
    print(k.name,v)

dw GGAAATTAGAGCTGGC
dw* GCCAGCTCTAATTTCC
dx GCGGCAGATAAATAGG
dx* CCTATTTATCTGCCGC
dx1m AGGCCCGAGAATTAAA
dx1m* TTTAATTCTCGGGCCT
dx2m GTAGGCTTGGTGATTG
dx2m* CAATCACCAAGCCTAC
dy1 GTTACATCTCCATCCG
dy1* CGGATGGAGATGTAAC
dy2 GTTCGGTTTAGTTTGC
dy2* GCAAACTAAACCGAAC


In [135]:
strands = my_results.to_analysis.strands
for k,v in strands.items():
    print(k.name,v)

base1 CGGATGGAGATGTAACCCTATTTATCTGCCGC
sy1 GTTACATCTCCATCCG
base2 GCAAACTAAACCGAACTTTAATTCTCGGGCCT
sy2 GTTCGGTTTAGTTTGC
base3 GCCAGCTCTAATTTCCCAATCACCAAGCCTAC
sw GGAAATTAGAGCTGGC
sx GCGGCAGATAAATAGG
sx1m AGGCCCGAGAATTAAA
sx2m GTAGGCTTGGTGATTG


In [ ]:
s = """
base1 CGGATGGAGATGTAACCCTATTTATCTGCCGC
sy1 GTTACATCTCCATCCG
base2 GCAAACTAAACCGAACTTTAATTCTCGGGCCT
sy2 GTTCGGTTTAGTTTGC
base3 GCCAGCTCTAATTTCCCAATCACCAAGCCTAC
sw GGAAATTAGAGCTGGC
sx GCGGCAGATAAATAGG
sx1m AGGCCCGAGAATTAAA
sx2m GTAGGCTTGGTGATTG
"""